In [12]:
import os
import tkinter as tk
from tkinter import ttk, messagebox
from db_utils import add_student, get_students, init_db, mark_attendance, get_attendance_history
from face_utils import Face
import cv2
import pickle
from db_utils import mark_attendance, get_students
import tkinter.messagebox as messagebox
import numpy as np

IMG_DIR = "student_images"

def gui_app():
    root = tk.Tk()
    root.title("Face Attendance System")
    root.geometry("750x480")

    notebook = ttk.Notebook(root)
    tab_students = ttk.Frame(notebook)
    tab_attendance = ttk.Frame(notebook)
    notebook.add(tab_students, text="Student Management")
    notebook.add(tab_attendance, text="Attendance")
    notebook.pack(expand=True, fill='both', padx=10, pady=10)

    #  STUDENT MANAGEMENT 

    tk.Label(tab_students, text="Student ID:").grid(row=0, column=0, sticky='e', padx=6, pady=6)
    tk.Label(tab_students, text="Full Name:").grid(row=1, column=0, sticky='e', padx=6, pady=6)
    tk.Label(tab_students, text="Class:").grid(row=2, column=0, sticky='e', padx=6, pady=6)
    sid_entry = tk.Entry(tab_students)
    name_entry = tk.Entry(tab_students)
    class_entry = tk.Entry(tab_students)
    sid_entry.grid(row=0, column=1, padx=6, pady=6)
    name_entry.grid(row=1, column=1, padx=6, pady=6)
    class_entry.grid(row=2, column=1, padx=6, pady=6)

    def capture_images():
        student_id = sid_entry.get().strip()
        if not student_id:
            messagebox.showwarning("Warning", "Please enter student ID before capturing images!")
            return
        face = Face(student_id=student_id)
        messagebox.showinfo("Instructions", "Position your face in front of the webcam, press 'q' to finish.")
        face.get_face()
        messagebox.showinfo("Success", f"Captured images for {student_id}.")

    def add():
        sid = sid_entry.get().strip()
        name = name_entry.get().strip()
        class_name = class_entry.get().strip()
        if not sid or not name or not class_name:
            messagebox.showwarning("Warning", "Please enter all required information!")
            return
        img_folder = os.path.join(IMG_DIR, sid)
        if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
            messagebox.showwarning("Warning", "You must capture images before saving the student!")
            return
        try:
            add_student(sid, name, class_name)
            messagebox.showinfo("Success", "Student added to the system.")
            refresh_students()
            sid_entry.delete(0, 'end')
            name_entry.delete(0, 'end')
            class_entry.delete(0, 'end')

        except Exception as e:
            messagebox.showerror("Error", str(e))

    cols = ("Student ID", "Full Name", "Class")
    tree = ttk.Treeview(tab_students, columns=cols, show='headings', height=9)
    for c in cols:
        tree.heading(c, text=c)
        tree.column(c, width=140)
    tree.grid(row=5, column=0, columnspan=2, pady=12)

    def refresh_students():
        for row in tree.get_children():
            tree.delete(row)
        for s in get_students():
            tree.insert('', 'end', values=s)
    refresh_students()

    tk.Button(tab_students, text="Capture Images", command=capture_images, width=14).grid(row=3, column=0, pady=8)
    tk.Button(tab_students, text="Save Student", command=add, width=14).grid(row=3, column=1, pady=8)

    #  ATTENDANCE

    def attendance_camera():
        model_path = "ml_face_person_identity.pkl"
        encoder_path = "Encode.pkl"
        try:
            ml_model = pickle.load(open(model_path, "rb"))
            le = pickle.load(open(encoder_path, "rb"))
        except FileNotFoundError:
            messagebox.showerror("Error", "Model or encoder not found. Please train the model first.")
            return

        proto = 'model/deploy.prototxt'
        face_model_file = 'model/res10_300x300_ssd_iter_140000_fp16.caffemodel'
        shape_model_file = 'model/openface.nn4.small2.v1.t7'
        model_face = cv2.dnn.readNetFromCaffe(proto, face_model_file)
        model_shape = cv2.dnn.readNetFromTorch(shape_model_file)

        cap = cv2.VideoCapture(0)
        cv2.namedWindow("Attendance", cv2.WINDOW_NORMAL)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            h, w = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 1.0, (300,300), (123,175,104), swapRB=False, crop=False)
            model_face.setInput(blob)
            faces = model_face.forward()

            if faces.shape[2] > 0:
                i = faces[0,0,:,2].argmax()
                confidence = faces[0,0,i,2]

                if confidence > 0.9:
                    box = faces[0,0,i,3:7] * np.array([w,h,w,h])
                    start_x, start_y, end_x, end_y = box.astype('int')
                    cv2.rectangle(frame, (start_x,start_y), (end_x,end_y), (0,255,0), 2)
                    roi = frame[start_y:end_y, start_x:end_x]
                    roi_blob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (255,255,255), swapRB=True, crop=True)
                    model_shape.setInput(roi_blob)
                    vectors = model_shape.forward()

                    face_name_label = ml_model.predict(vectors)[0]
                    student_id_pred = le.inverse_transform([face_name_label])[0]
                    face_name_score = ml_model.predict_proba(vectors).max()

                    if face_name_score > 0.7:
                        mark_attendance(student_id_pred)
                        name_show = student_id_pred
                        for row in get_students():
                            if row[0] == student_id_pred:
                                name_show = f"{row[1]} ({student_id_pred})"
                                break
                        cv2.putText(frame, f"Welcome: {name_show}", (40,120), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 3)
                        cv2.imshow("Attendance", frame)
                        cv2.waitKey(1600)
                        messagebox.showinfo("Attendance Successful", f"Attendance recorded for: {name_show}")
                        break

            cv2.imshow("Attendance", frame)
            if cv2.waitKey(10) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


    def show_attendance_history():
        rows = get_attendance_history()
        if not rows:
            messagebox.showinfo("Attendance History", "No attendance data found.")
            return

        history_window = tk.Toplevel()
        history_window.title("Attendance History")
        history_window.geometry("700x350")
        cols = ("Date", "Student ID", "Full Name", "Class", "Status")
        tree = ttk.Treeview(history_window, columns=cols, show='headings', height=15)
        for c in cols:
            tree.heading(c, text=c)
            tree.column(c, width=120)
        tree.pack(expand=True, fill='both')

        for row in rows:
            tree.insert('', 'end', values=row)

    tk.Button(tab_attendance, text="Camera Attendance", font=("Arial", 13), command=attendance_camera, width=24)\
        .pack(pady=50)
    tk.Button(tab_attendance, text="View Attendance History", font=("Arial", 12), command=show_attendance_history, width=24)\
        .pack(pady=5)

    root.mainloop()

if __name__ == "__main__":
    os.makedirs(IMG_DIR, exist_ok=True)
    init_db()
    gui_app()
